In [ ]:
import numpy as np
import torch

In [ ]:
from transformers import BertTokenizer, AutoModel

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model = AutoModel.from_pretrained(model_name)

In [ ]:
def embed_tokens(input_ids, attention_mask):
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    # Use the [CLS] token representation for sentence-level embedding
    embeddings = outputs.last_hidden_state[:, 0, :]  # Shape: [batch_size, hidden_dim]
    return embeddings

In [ ]:
all_embeddings = torch.load("embeddings.pt")

C:\Users\ObelWork-1\AppData\Local\Temp\ipykernel_6284\64800262.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  all_embeddings = torch.load("embeddings.pt")


In [ ]:
embeddings = all_embeddings.numpy()

In [ ]:
import json

# Load JSON data
with open('C:/Users/ObelWork-1/Downloads/data_for_vectordb.json', 'r') as file:
    data = json.load(file)


cve_ids = [item['id'] for item in data]
cve_ids = np.array(cve_ids)

In [ ]:
from pinecone import Pinecone

pc = Pinecone(api_key="2f5652db-c1bc-4529-a189-84e3602bf46c")
index = pc.Index("cveindex")

In [ ]:
for i, cve_id in enumerate(cve_ids):
  if cve_id:
    vector = [{"id":cve_ids[i],"values":embeddings[i].tolist()}]
    index.upsert(vectors=vector, namespace= "ns1")`

## Testing

In [ ]:
query = "An authenticated user may be able to misuse parameters to inject arbitrary operating system commands into mySCADA myPRO versions 8.25.0 and prior."

In [ ]:
query_encoded_inputs = tokenizer(query, padding=True, truncation=True, return_tensors='pt')

In [ ]:
query_token_ids = query_encoded_inputs['input_ids']
query_attention_masks = query_encoded_inputs['attention_mask']

In [ ]:
query_embeddings = embed_tokens(query_token_ids, query_attention_masks)

In [ ]:
query_vector = query_embeddings.tolist()

In [ ]:
query_vector = query_vector[0]


In [ ]:
response = index.query(
    namespace="ns1",
    vector=query_vector,
    top_k=5,
    include_values=True,
)



In [ ]:
for res in response['matches']:
  print(res['id'])

CVE-2022-0999
CVE-2023-28384
CVE-2023-28400
CVE-2023-28716
CVE-2023-29150
